In [1]:
import imageio
import random
import numpy as np  
import pandas as pd 

from cellpose import models
from cellpose.utils import fill_holes_and_remove_small_masks
from cellstitch.utils import *
from cellstitch.pipeline import *
from cellstitch.evaluation import *

In [2]:
filenames = set(get_filenames("../DATA/ATAS/raw/"))

In [3]:
# test_filenames = set(random.sample(filenames, int(0.3 * len(filenames))))
# train_filenames = filenames - test_filenames

### Train cellpose model from scratch

In [4]:
# generate training data
data_folder = "../DATA/ATAS"
cellpose_folder = "../DATA/ATAS/cellpose_train"

In [5]:
for train_filename in train_filenames: 
    img = np.load("%s/raw/%s" % (data_folder, train_filename))
    labels = np.load("%s/labels/%s" % (data_folder, train_filename)) 
    depth = img.shape[0] 
    
    for i in range(depth): 
        imageio.imwrite("%s/%s_%s.tif" % (cellpose_folder, train_filename, i), img[i])
        imageio.imwrite("%s/%s_%s_masks.tif" % (cellpose_folder, train_filename, i), labels[i])

NameError: name 'train_filenames' is not defined

`python -m cellpose --train --use_gpu --dir ../DATA/ATAS/cellpose_train --pretrained_model None --n_epochs 100  --verbose`

### Test file names

In [2]:
filenames = set(get_filenames("../DATA/ATAS/raw/"))

In [3]:
train_filenames = []

In [4]:
for file in os.listdir("../DATA/ATAS/cellpose_train/"):
    if file.endswith(".npy_95.tif"):
        train_filenames.append(file[:-7])

In [5]:
test_filenames = filenames - set(train_filenames)

In [33]:
dataset = "ATAS_10" # ATAS, ATAS_5, ATAS_10

### Generate cellpose3d results

In [34]:
model_dir = '../DATA/ATAS/cellpose_train/models/cellpose_residual_on_style_on_concatenation_off_cellpose_train_2023_04_28_14_46_49.933105'

In [35]:
flow_threshold = 1
model = models.CellposeModel(gpu=True, pretrained_model=model_dir)

In [36]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename)) 
    masks, _, _ = model.eval(img, do_3D=True, flow_threshold=flow_threshold, channels = [0,0]) 
    np.save("./results/%s/cellpose3d/%s" % (dataset, test_filename), masks)

Starting 32hrs_plant13_trim-acylYFP.npy
Starting 40hrs_plant13_trim-acylYFP.npy
Starting 36hrs_plant15_trim-acylYFP.npy
Starting 20hrs_plant2_trim-acylYFP.npy
Starting 44hrs_plant2_trim-acylYFP.npy
Starting 48hrs_plant4_trim-acylYFP.npy
Starting 8hrs_plant1_trim-acylYFP.npy
Starting 84hrs_plant18_trim-acylYFP.npy
Starting 0hrs_plant2_trim-acylYFP.npy
Starting 24hrs_plant15_trim-acylYFP.npy
Starting 80hrs_plant15_trim-acylYFP.npy
Starting 12hrs_plant18_trim-acylYFP.npy
Starting 0hrs_plant13_trim-acylYFP.npy
Starting 4hrs_plant15_trim-acylYFP.npy
Starting 20hrs_plant13_trim-acylYFP.npy
Starting 36hrs_plant2_trim-acylYFP.npy
Starting 52hrs_plant2_trim-acylYFP.npy
Starting 60hrs_plant2_trim-acylYFP.npy
Starting 8hrs_plant15_trim-acylYFP.npy
Starting 44hrs_plant18_trim-acylYFP.npy
Starting 40hrs_plant1_trim-acylYFP.npy
Starting 44hrs_plant4_trim-acylYFP.npy
Starting 72hrs_plant2_trim-acylYFP.npy
Starting 24hrs_plant13_trim-acylYFP.npy
Starting 0hrs_plant18_trim-acylYFP.npy
Starting 40hrs_pl

### Generate cellstitch results

In [37]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename))
    
    cellstitch, _, _ = model.eval(list(img), flow_threshold=flow_threshold, channels = [0,0])
    cellstitch = np.array(cellstitch)

    yz_masks, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold, channels = [0,0])
    yz_masks = np.array(yz_masks).transpose(1,0,2)

    xz_masks, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold, channels = [0,0])
    xz_masks = np.array(xz_masks).transpose(2,1,0)

    full_stitch(cellstitch, yz_masks, xz_masks)
    
    np.save("./results/%s/cellstitch/%s" % (dataset, test_filename), cellstitch) 

Starting 32hrs_plant13_trim-acylYFP.npy


Starting 40hrs_plant13_trim-acylYFP.npy


Starting 36hrs_plant15_trim-acylYFP.npy


Starting 20hrs_plant2_trim-acylYFP.npy


Starting 44hrs_plant2_trim-acylYFP.npy


...

Starting 60hrs_plant4_trim-acylYFP.npy


Starting 76hrs_plant1_trim-acylYFP.npy


Starting 64hrs_plant13_trim-acylYFP.npy


Starting 68hrs_plant1_trim-acylYFP.npy


Starting 12hrs_plant2_trim-acylYFP.npy


### Generate cellstitch3d results

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename))
    
    cellstitch = np.load("./results/%s/cellpose3d/%s" % (dataset, test_filename))
    
    yz_masks, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold, channels = [0,0])
    yz_masks = np.array(yz_masks).transpose(1,0,2)

    xz_masks, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold, channels = [0,0])
    xz_masks = np.array(xz_masks).transpose(2,1,0)
    
    full_stitch(cellstitch, yz_masks, xz_masks)
    
    np.save("./results/%s/cellstitch3d/%s" % (dataset, test_filename), cellstitch)

Starting 32hrs_plant13_trim-acylYFP.npy


Starting 40hrs_plant13_trim-acylYFP.npy


Starting 36hrs_plant15_trim-acylYFP.npy


# Benchmark Results

In [ ]:
ap_threshold = 0.5

In [ ]:
methods = ["cellpose3d", "cellstitch", "cellstitch3d"]

for method in methods: 
    data = [] 
    for filename in test_filenames:
        print("Starting %s" % filename)
        labels = np.load('../DATA/%s/labels/%s' % (dataset, filename))
        true_num_cells = np.unique(labels).size - 1 
        true_avg_vol = get_avg_vol(labels) 

        masks = np.load("./results/%s/%s/%s" % (dataset, method, filename))

        num_cells = np.unique(masks).size - 1
        d_num_cells = abs(num_cells - true_num_cells) / true_num_cells

        avg_vol = get_avg_vol(masks)
        d_avg_vol = abs(true_avg_vol - avg_vol) / true_avg_vol

        ap, tp, fp, fn = average_precision(labels, masks, ap_threshold)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
    
        row = [
            filename, 
            d_num_cells, 
            d_avg_vol, 
            ap, 
            tp, 
            fp, 
            fn, 
            precision,
            recall, 
            true_num_cells, 
            true_avg_vol, 
            num_cells, 
            avg_vol
        ]

        data.append(row)

    df = pd.DataFrame(data, columns=[
        "filename",  
        "d_num_cells", 
        "d_avg_vol", 
        "ap", 
        "tp", 
        "fp", 
        "fn",
        "precision",
        "recall",
        "true_num_cells", 
        "true_avg_vol", 
        "num_cells", 
        "avg_vol"
    ])

    df.to_csv("./results/%s/%s.csv" % (dataset, method), index=False)